This script evaluates the DEFAME variants/models that are evaluated on the gaza-israel dataset. It combines the DEFAME outputs that are saved in the DEFAME/out folder and the results from my own qualitative analyses. The output folders for each model variant are copied from the DEFAME output folder in this evaluation folder for a better overview. The results from the qualitative analyses were documented in Notion and stored in csv files to easily import them here.



#### 1) Evaluation Nr.1: 

- Model 1, Run 1, Folder name in DEFAME/out directory: 2025-07-20_09-52 default

- Original folder path in DEFAME out directory: "Practical Implementation/DEFAME/out/gaza_israel/gaza_israel/summary/dynamic/gemini-2.0-flash-lite/2025-07-20_09-52 default"

- New folder path with a copy of the original folder: "Practical Implementation/evaluation/gaza_israel/model_1_run_1_(2025-07-20_09-52 default)/DEFAME_output/2025-07-20_09-52 default"

In [1]:
## import relevant libraries

import pandas as pd
import json

##### 1.1) Show model configuration 

In [4]:
##### 1.1) Show model configuration

with open('model1_run1_(2025-07-20_09-52 default)/DEFAME_output/2025-07-20_09-52 default/config.yaml', 'r') as file:
    content = file.read()
    print(content)

allowed_actions:
- web_search
- image_search
- reverse_search
- geolocate
benchmark_kwargs:
  variant: dev
benchmark_name: gaza_israel
continue_experiment_dir: null
experiment_name: default
fact_checker_kwargs:
  decompose: false
  decontextualize: false
  filter_check_worthy: false
  interpret: false
  max_iterations: 3
  max_result_len: 64000
  procedure_variant: summary/dynamic
llm: gemini-2.0-flash-lite
llm_kwargs:
  temperature: 0.01
n_samples: null
n_workers: 2
print_log_level: info
random_sampling: false
sample_ids: null
tools_config:
  geolocator: null
  searcher:
    limit_per_search: 3
    search_engine_config:
      google: null
      google_vision: null



- Note: The variant is set to "dev" in the benchmark_kwargs parameter although the dataset is a test set and not a dev set. This is due to the fact that the original DEFAME implementation only creates the confusion matrix (seen below) for "dev" datasets. Since the confusion matrix is necessary to qualitative analyze the mispredictions of the model and I didn't want to adjust the entire code, I'm using this parameter value. 

##### 1.2) Show results: 

a) confusion matrix


![Confusion_matrix](model_1_run_1_(2025-07-20_09-52%20default)/DEFAME_output/2025-07-20_09-52%20default/confusion.png)

b) evaluation metrics

In [6]:
## b) evaluation metrics

# Open and read the JSON file with model results
with open('model1_run1_(2025-07-20_09-52 default)/DEFAME_output/2025-07-20_09-52 default/results.json', 'r') as file:
    model_results = json.load(file)
    # for key in model_results.keys():
    #     print(key)

# Print the data in a readable format
print(json.dumps(model_results, indent=4))

{
    "Number of workers": 2,
    "Total run duration": 6023.802916526794,
    "Time per claim": 107.98897849670564,
    "Model": {
        "Calls": 1167,
        "Input tokens": 5834139.0,
        "Output tokens": 112164,
        "Input tokens cost": 0.4375604249999979,
        "Output tokens cost": 0.0336491999999974,
        "Total cost": 0.4712096249999977
    },
    "Tools": {
        "searcher": {
            "Total searches": 206,
            "Search engine calls": {
                "google": 151,
                "google_vision": 55
            }
        }
    },
    "Predictions": {
        "Total": 99,
        "Refused": 0,
        "Metrics": {
            "FALSE_Precision": 0.867,
            "FALSE_Recall": 0.441,
            "FALSE_F1_Score": 0.584,
            "MISLEADING_Precision": 0.512,
            "MISLEADING_Recall": 0.7,
            "MISLEADING_F1_Score": 0.592,
            "NEI_Precision": 0.111,
            "NEI_Recall": 0.25,
            "NEI_F1_Score": 0.154,
  

- Note: The listed model costs are rough estimates based on the default DEFAME calculations. They are mainly based on OpenAI's token counts and calculations and not on Google's. I have not updated it, because I'm getting the model costs on the Google Cloud Console anyway. I will report the actual costs per evaluated model variant in the thesis. 

##### 1.3) Concatenate/merge the csv with the model prediction results ("predictions_csv") with the final gaza-israel dataset.  

In [7]:
## import model prediction results

df_model_prediction_results = pd.read_csv("model1_run1_(2025-07-20_09-52 default)/DEFAME_output/2025-07-20_09-52 default/predictions.csv")
print(len(df_model_prediction_results))
print(df_model_prediction_results.dtypes)
df_model_prediction_results.head()

99
sample_index         int64
claim               object
target              object
predicted           object
justification       object
correct               bool
gt_justification    object
dtype: object


,sample_index,claim,target,predicted,justification,correct,gt_justification
0,0,"<image:1> This image shows General John Pagri,...",FALSE,MISLEADING,The image in the claim is a photograph of two ...,False,A reverse image search reveals the photograph ...
1,1,<image:2> This image shows the ruins after a 7...,FALSE,TRUE,The image depicts the aftermath of the 2025 My...,False,"However, a reverse image search for the first ..."
2,2,<image:3> This is an authentic image of a moth...,FALSE,FALSE,The claim is false. The image of the woman hol...,True,A keyword search found a wider-angle version o...
3,3,<image:4> This image shows Israelis who have s...,FALSE,FALSE,"The claim is false. Multiple sources, includin...",True,A reverse image search via Google traced the v...
4,4,<image:5> This image shows the U.S. bombarding...,FALSE,MISLEADING,The claim is misleading. While the U.S. did st...,False,But a reverse image search on Google found the...


In [8]:
## import final dataset 

df_gaza_israel_final = pd.read_csv("../../DEFAME/data/gaza_israel/gaza_israel_dataset_combined_010724_300425_final.csv")
print(len(df_gaza_israel_final))
print(df_gaza_israel_final.columns)
print(df_gaza_israel_final.dtypes)
df_gaza_israel_final.head()

100
Index(['Unnamed: 0', 'id', 'Website', 'Article_URL', 'Headline', 'Claim_Date',
       'Review_Date', 'Query/Keyword', 'Original_Claim_Website',
       'Original_Claim_Only', 'Claim', 'Image_URL', 'Image_Path',
       'Label_Website', 'Label', 'Context/Label_Explanation',
       'Text_Only_Claim', 'Normal_Image', 'AI_Generated_Image',
       'Altered_Image', 'Data_Collection_Type'],
      dtype='object')
Unnamed: 0                    int64
id                            int64
Website                      object
Article_URL                  object
Headline                     object
Claim_Date                   object
Review_Date                  object
Query/Keyword                object
Original_Claim_Website       object
Original_Claim_Only          object
Claim                        object
Image_URL                    object
Image_Path                   object
Label_Website                object
Label                        object
Context/Label_Explanation    object
Text_Only_Cla

,Unnamed: 0,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Original_Claim_Website,Original_Claim_Only,...,Image_URL,Image_Path,Label_Website,Label,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type
0,0,0,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.42JE3T2,False claims about US general being killed in ...,2025-04-06 0:00:00,2025-04-17 0:00:00,Israeli-Palestinian conflict,"""BREAKING: Reports confirm the death of Genera...","""BREAKING: Reports confirm the death of Genera...",...,https://pbs.twimg.com/media/Gn4wjgBXoAALax1?fo...,images/gaza_israel/0.jpg,FALSE,False,A reverse image search reveals the photograph ...,False,True,False,False,Manual
1,1,1,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.39432WD,"Pictures show Gaza in ruins, not Myanmar quake...",2025-03-29 0:00:00,2025-04-17 0:00:00,Israeli-Palestinian conflict,"""A 7.9-magnitude earthquake in Myanmar, devast...","""A 7.9-magnitude earthquake in Myanmar, devast...",...,https://web.archive.org/web/20250402070342im_/...,images/gaza_israel/1.jpg,FALSE,False,"However, a reverse image search for the first ...",False,True,False,False,Manual
2,2,2,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.37W26WD,Image of 'Gazan mother with son's bones' is ma...,2025-03-24 0:00:00,2025-03-27 0:00:00,Israeli-Palestinian conflict,"""A mother holds the bones of her son in Gaza. ...","""A mother holds the bones of her son in Gaza. ...",...,https://web.archive.org/web/20250527092039/htt...,images/gaza_israel/2.jpg,FALSE,False,A keyword search found a wider-angle version o...,False,False,True,False,Manual
3,3,3,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36XY2FV,Old image misrepresented as 'Israeli settlers'...,2025-02-16 0:00:00,2025-02-19 0:00:00,Israeli-Palestinian conflict,"""Israeli media reports there are Jews who have...","""Israeli media reports there are Jews who have...",...,https://pbs.twimg.com/media/Gj5tbXiaQAAoG1z?fo...,images/gaza_israel/3.jpg,FALSE,False,A reverse image search via Google traced the v...,False,True,False,False,Manual
4,4,4,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36RF6BZ,"Picture shows Israel's 2018 Gaza attack, not a...",2024-12-22 0:00:00,2024-12-26 0:00:00,Israeli-Palestinian conflict,"""Americans bombarded Yemen in reprisal after Y...","""Americans bombarded Yemen in reprisal after Y...",...,https://media.gettyimages.com/id/1060718622/de...,images/gaza_israel/4.jpg,FALSE,False,But a reverse image search on Google found the...,False,True,False,False,Manual


In [9]:
## The df_model_prediction_results has 99 rows, yet the df_gaza_israel_final has 100 rows. Let's find out for which claim the model did not predict a label.

# Get the set of ids in each df
ids_final = set(df_gaza_israel_final['id'])
ids_model = set(df_model_prediction_results['sample_index'])

# Find which id is in final but NOT in model predictions
missing_id = ids_final - ids_model

print("Missing id (from model predictions):", missing_id)

Missing id (from model predictions): {94}


- Looking into the output folder, in the folder "94" there are only "log.txt" and "model_communication.txt" files, but no reports in pdf/md were created. Looking into the two text files, it seems that the model performed all necessary actions, created a summary but did not yield a prediction. It's not clear why. 

In [10]:
# Columns that are necessary from df with model prediction results: "predicted", "justification", "correct"

## 1) Rename the id column in df_model_prediction_results for merge/inner join

df_model_prediction_results_renamed = df_model_prediction_results.rename(columns={"sample_index": "id"})
#print(df_model_prediction_results.dtypes)


## 2) Perform the inner join

df_gaza_israel_merged = pd.merge(df_gaza_israel_final, df_model_prediction_results_renamed[['id', 'predicted', 'justification', 'correct']], on = "id", how = "inner")
df_gaza_israel_merged.head()



,Unnamed: 0,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Original_Claim_Website,Original_Claim_Only,...,Label,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,predicted,justification,correct
0,0,0,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.42JE3T2,False claims about US general being killed in ...,2025-04-06 0:00:00,2025-04-17 0:00:00,Israeli-Palestinian conflict,"""BREAKING: Reports confirm the death of Genera...","""BREAKING: Reports confirm the death of Genera...",...,False,A reverse image search reveals the photograph ...,False,True,False,False,Manual,MISLEADING,The image in the claim is a photograph of two ...,False
1,1,1,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.39432WD,"Pictures show Gaza in ruins, not Myanmar quake...",2025-03-29 0:00:00,2025-04-17 0:00:00,Israeli-Palestinian conflict,"""A 7.9-magnitude earthquake in Myanmar, devast...","""A 7.9-magnitude earthquake in Myanmar, devast...",...,False,"However, a reverse image search for the first ...",False,True,False,False,Manual,TRUE,The image depicts the aftermath of the 2025 My...,False
2,2,2,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.37W26WD,Image of 'Gazan mother with son's bones' is ma...,2025-03-24 0:00:00,2025-03-27 0:00:00,Israeli-Palestinian conflict,"""A mother holds the bones of her son in Gaza. ...","""A mother holds the bones of her son in Gaza. ...",...,False,A keyword search found a wider-angle version o...,False,False,True,False,Manual,FALSE,The claim is false. The image of the woman hol...,True
3,3,3,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36XY2FV,Old image misrepresented as 'Israeli settlers'...,2025-02-16 0:00:00,2025-02-19 0:00:00,Israeli-Palestinian conflict,"""Israeli media reports there are Jews who have...","""Israeli media reports there are Jews who have...",...,False,A reverse image search via Google traced the v...,False,True,False,False,Manual,FALSE,"The claim is false. Multiple sources, includin...",True
4,4,4,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36RF6BZ,"Picture shows Israel's 2018 Gaza attack, not a...",2024-12-22 0:00:00,2024-12-26 0:00:00,Israeli-Palestinian conflict,"""Americans bombarded Yemen in reprisal after Y...","""Americans bombarded Yemen in reprisal after Y...",...,False,But a reverse image search on Google found the...,False,True,False,False,Manual,MISLEADING,The claim is misleading. While the U.S. did st...,False


##### 1.4) Analysis of wrong predictions of the model

1.4.1) Quantitative Analysis

In [11]:
## 1st Question: What is the ratio of correct/wrong predictions for the different claim types (text_only claims, claims with normal images, claims with AI images, claims with altered images)

## Look at claim type distribution again

print(df_gaza_israel_merged["Text_Only_Claim"].value_counts())
print(df_gaza_israel_merged["Normal_Image"].value_counts())
print(df_gaza_israel_merged["AI_Generated_Image"].value_counts())
print(df_gaza_israel_merged["Altered_Image"].value_counts())

Text_Only_Claim
False    64
True     35
Name: count, dtype: int64
Normal_Image
False    51
True     48
Name: count, dtype: int64
AI_Generated_Image
False    90
True      9
Name: count, dtype: int64
Altered_Image
False    92
True      7
Name: count, dtype: int64


In [12]:
### 1) Text-only claims

## Correct vs. wrong predictions for all text-only claims

all_text_only_claims = df_gaza_israel_merged[df_gaza_israel_merged["Text_Only_Claim"] == True]

## General distribution

text_only_counts = all_text_only_claims["correct"].value_counts()
text_only_percentages = all_text_only_claims["correct"].value_counts(normalize=True) * 100
print("Text-only Claims: Correct vs Wrong Predictions (overall):\n")
print("Absolute numbers:")
print(text_only_counts)
print("\n")
print("Relative numbers:")
print(text_only_percentages)

## Distribution per fact-checking website -> any pattern?

result = all_text_only_claims.groupby(["correct", "Website"]).size()
print("\nText-Only claims: Correct vs Wrong Predictions (per Website):\n")
print(result)


## Distribution per predicted label and ground truth label -> any pattern?

grouped_predlabel_gtlabel = all_text_only_claims.groupby(['correct', 'predicted', 'Label']).size()
print("\nText-Only claims: Correct vs Wrong Predictions (per predicted label and ground truth label):\n")
print(grouped_predlabel_gtlabel)

## Distribuiton per predicted label, ground truth label and website -> any pattern?

grouped_predlabel_gtlabel_website = all_text_only_claims.groupby(['correct', 'predicted', 'Label', 'Website']).size()
print("\nText-Only claims: Correct vs Wrong Predictions (per predicted label, ground truth label and website):\n")
print(grouped_predlabel_gtlabel_website)


Text-only Claims: Correct vs Wrong Predictions (overall):

Absolute numbers:
correct
True     21
False    14
Name: count, dtype: int64


Relative numbers:
correct
True     60.0
False    40.0
Name: proportion, dtype: float64

Text-Only claims: Correct vs Wrong Predictions (per Website):

correct  Website          
False    AFP Factcheck        3
         Politifact           2
         Reuters              3
         Snopes               1
         checkyourfact.com    2
         misbar.com           2
         usatoday.com         1
True     AFP Factcheck        1
         Politifact           2
         Reuters              6
         Snopes               8
         misbar.com           2
         newsmeter.in         1
         usatoday.com         1
dtype: int64

Text-Only claims: Correct vs Wrong Predictions (per predicted label and ground truth label):

correct  predicted   Label     
False    MISLEADING  False          6
                     NEI            1
         NEI         

##### Analysis for Predictions per Website

- Seems like the predictions for text-only-claims works particularly well for claims from Snopes (8 of 9) and Reuters (6 of 9)
- Seems like the predictions for text-only-claims works bad for claims from AFP Factcheck (1 of 4) and checkyourfact.com (0 of 2) 
- Seems like the predictions for text-only-claims yields mixed results for the remaining websites, e.g., Politifact (2 of 4), misbar.com (2 of 4), usatoday.com (1 of 2)


In [13]:
### 2) Claims with normal image

## Correct vs. wrong predictions for all claims with normal images

all_claims_normal_image = df_gaza_israel_merged[df_gaza_israel_merged["Normal_Image"] == True]

## General distribution

claims_normal_image_counts = all_claims_normal_image["correct"].value_counts()
claims_normal_image_percentages = all_claims_normal_image["correct"].value_counts(normalize=True) * 100
print("Claims with normal images: Correct vs Wrong Predictions (overall):\n")
print("Absolute numbers:")
print(claims_normal_image_counts)
print("\n")
print("Relative numbers:")
print(claims_normal_image_percentages)

## Distribution per fact-checking website -> any pattern?

grouped_website = all_claims_normal_image.groupby(["correct", "Website"]).size()
print("\nClaims with normal images: Correct vs Wrong Predictions (per Website):\n")
print(grouped_website)


## Distribution per predicted label and ground truth label -> any pattern?

grouped_predlabel_gtlabel = all_claims_normal_image.groupby(['correct', 'predicted', 'Label']).size()
print("\nClaims with normal images: Correct vs Wrong Predictions (per predicted label and ground truth label):\n")
print(grouped_predlabel_gtlabel)

## Distribuiton per predicted label, ground truth label and website -> any pattern?

grouped_predlabel_gtlabel_website = all_claims_normal_image.groupby(['correct', 'predicted', 'Label', 'Website']).size()
print("\nClaims with normal images: Correct vs Wrong Predictions (per predicted label, ground truth label and website):\n")
print(grouped_predlabel_gtlabel_website)


Claims with normal images: Correct vs Wrong Predictions (overall):

Absolute numbers:
correct
True     25
False    23
Name: count, dtype: int64


Relative numbers:
correct
True     52.083333
False    47.916667
Name: proportion, dtype: float64

Claims with normal images: Correct vs Wrong Predictions (per Website):

correct  Website           
False    AFP Factcheck          8
         Politifact             1
         Reuters                1
         checkyourfact.com      1
         logicallyfacts.com     3
         misbar.com             9
True     AFP Factcheck          3
         Reuters                7
         Snopes                 2
         logicallyfacts.com     1
         misbar.com            10
         newsmeter.in           1
         usatoday.com           1
dtype: int64

Claims with normal images: Correct vs Wrong Predictions (per predicted label and ground truth label):

correct  predicted   Label     
False    FALSE       Misleading     4
         MISLEADING  False 

In [14]:
### 3) Claims with AI generated image

## Correct vs. wrong predictions for all claims with AI images

all_claims_ai_image = df_gaza_israel_merged[df_gaza_israel_merged["AI_Generated_Image"] == True]

## General distribution

claims_ai_image_counts = all_claims_ai_image["correct"].value_counts()
claims_ai_image_percentages = all_claims_ai_image["correct"].value_counts(normalize=True) * 100
print("Claims with AI images: Correct vs Wrong Predictions (overall):\n")
print("Absolute numbers:")
print(claims_ai_image_counts)
print("\n")
print("Relative numbers:")
print(claims_ai_image_percentages)


## Distribution per fact-checking website -> any pattern?

result = all_claims_ai_image.groupby(["correct", "Website"]).size()
print("\nClaims with AI images: Correct vs Wrong Predictions (per Website):\n")
print(result)

## Distribution per predicted label and ground truth label -> any pattern?

grouped_predlabel_gtlabel = all_claims_ai_image.groupby(['correct', 'predicted', 'Label']).size()
print("\nClaims with AI images: Correct vs Wrong Predictions (per predicted label and ground truth label):\n")
print(grouped_predlabel_gtlabel)

## Distribuiton per predicted label, ground truth label and website -> any pattern?

grouped_predlabel_gtlabel_website = all_claims_ai_image.groupby(['correct', 'predicted', 'Label', 'Website']).size()
print("\nClaims with AI images: Correct vs Wrong Predictions (per predicted label, ground truth label and Website):\n")
print(grouped_predlabel_gtlabel_website)


Claims with AI images: Correct vs Wrong Predictions (overall):

Absolute numbers:
correct
True     6
False    3
Name: count, dtype: int64


Relative numbers:
correct
True     66.666667
False    33.333333
Name: proportion, dtype: float64

Claims with AI images: Correct vs Wrong Predictions (per Website):

correct  Website           
False    AFP Factcheck         1
         logicallyfacts.com    1
         misbar.com            1
True     AFP Factcheck         2
         Reuters               3
         newsmeter.in          1
dtype: int64

Claims with AI images: Correct vs Wrong Predictions (per predicted label and ground truth label):

correct  predicted   Label     
False    MISLEADING  False         1
         NEI         False         2
True     FALSE       False         5
         MISLEADING  Misleading    1
dtype: int64

Claims with AI images: Correct vs Wrong Predictions (per predicted label, ground truth label and Website):

correct  predicted   Label       Website           
F

In [15]:
### 4) Claims with altered images

## Correct vs. wrong predictions for all claims with altered images

all_claims_altered_image = df_gaza_israel_merged[df_gaza_israel_merged["Altered_Image"] == True]

## General distribution

claims_altered_image_counts = all_claims_altered_image["correct"].value_counts()
claims_altered_image_percentages = all_claims_altered_image["correct"].value_counts(normalize=True) * 100

# Order the counts to also show "True" before "False" values (by default it is ordered by frequency. For the claims with altered images this results in printing out "False" values first).
## Thus, it is consistent with all other claim types
claims_altered_image_counts_ordered = claims_altered_image_counts.reindex([True, False])
claims_altered_image_percentages_ordered = claims_altered_image_percentages.reindex([True, False])

print("Claims with altered images: Correct vs Wrong Predictions (overall):\n")
print("Absolute numbers:")
print(claims_altered_image_counts_ordered)
print("\n")
print("Relative numbers:")
print(claims_altered_image_percentages_ordered)


## Distribution per fact-checking website -> any pattern?

grouped_website = all_claims_altered_image.groupby(["correct", "Website"]).size()
print("\nClaims with altered images: Correct vs Wrong Predictions (per Website):\n")
print(grouped_website)


## Distribution per predicted label and ground truth label -> any pattern?

grouped_predlabel_gtlabel = all_claims_altered_image.groupby(['correct', 'predicted', 'Label']).size()
print("\nClaims with altered images: Correct vs Wrong Predictions (per predicted label and ground truth label):\n")
print(grouped_predlabel_gtlabel)

## Distribuiton per predicted label, ground truth label and website -> any pattern?

grouped_predlabel_gtlabel_website = all_claims_altered_image.groupby(['correct', 'predicted', 'Label', 'Website']).size()
print("\nClaims with altered images: Correct vs Wrong Predictions (per predicted label, ground truth label and Website):\n")
print(grouped_predlabel_gtlabel_website)


Claims with altered images: Correct vs Wrong Predictions (overall):

Absolute numbers:
correct
True     2
False    5
Name: count, dtype: int64


Relative numbers:
correct
True     28.571429
False    71.428571
Name: proportion, dtype: float64

Claims with altered images: Correct vs Wrong Predictions (per Website):

correct  Website           
False    Reuters               2
         Snopes                1
         logicallyfacts.com    1
         misbar.com            1
True     logicallyfacts.com    1
         usatoday.com          1
dtype: int64

Claims with altered images: Correct vs Wrong Predictions (per predicted label and ground truth label):

correct  predicted   Label
False    MISLEADING  False    3
         TRUE        False    2
True     FALSE       False    2
dtype: int64

Claims with altered images: Correct vs Wrong Predictions (per predicted label, ground truth label and Website):

correct  predicted   Label  Website           
False    MISLEADING  False  Snopes         

In [16]:
# Save the model performance per claim type in a dictionary and print a readable summary as an overview


# Store model results in a structured dictionary
model_results_claim_type = {
    "Text-Only Claims": {
        "Absolute": text_only_counts.to_dict(),
        "Percentage": {k: f"{v:.1f}%" for k, v in text_only_percentages.to_dict().items()}
    },
    "Claims with Normal Images": {
        "Absolute": claims_normal_image_counts.to_dict(),
        "Percentage": {k: f"{v:.1f}%" for k, v in claims_normal_image_percentages.to_dict().items()}
    },
    "Claims with AI Images": {
        "Absolute": claims_ai_image_counts.to_dict(),
        "Percentage": {k: f"{v:.1f}%" for k, v in claims_ai_image_percentages.to_dict().items()}
    },
    "Claims with Altered Images": {
        "Absolute": claims_altered_image_counts_ordered.to_dict(),
        "Percentage": {k: f"{v:.1f}%" for k, v in claims_altered_image_percentages_ordered.to_dict().items()}
    },
}

# Print results in a readable format
print("Summary of Model Performance by Claim Type\n")
print("=" * 50)

for claim_type, data in model_results_claim_type.items():
    print(f"\n{claim_type}:")
    print("-" * 30)
    for status, counts in data["Absolute"].items():
        percentage = data["Percentage"][status]
        # Map boolean values to strings
        status_labels = {True: "True", False: "False"}
        label = status_labels[status]
        print(f"  {label}: {counts} ({percentage})")


Summary of Model Performance by Claim Type


Text-Only Claims:
------------------------------
  True: 21 (60.0%)
  False: 14 (40.0%)

Claims with Normal Images:
------------------------------
  True: 25 (52.1%)
  False: 23 (47.9%)

Claims with AI Images:
------------------------------
  True: 6 (66.7%)
  False: 3 (33.3%)

Claims with Altered Images:
------------------------------
  True: 2 (28.6%)
  False: 5 (71.4%)


In [19]:
# store the model results in a df and save it in the folder of model_1_run1

df_model_results = pd.DataFrame.from_dict(data = model_results_claim_type)
df_model_results

df_model_results.to_csv("model1_run1_(2025-07-20_09-52 default)/model_1_run1_results_claimtype.csv")

1.4.2) Qualitative Analysis

#### Qualitative Analyis Part 1 (n = 13): 

- Prediction = True
- Ground Truth Label = False/Misleading/True

In [20]:
df_qualitative_analysis_part_1 = pd.read_csv("model1_run1_(2025-07-20_09-52 default)/own_qualitative_analysis/model1_run1_qualitative_analysis_false_predictions_part1_n13.csv")
print(len(df_qualitative_analysis_part_1))
print(df_qualitative_analysis_part_1.dtypes)
df_qualitative_analysis_part_1.head()

13
Sample_index                              int64
Claim                                    object
Fact-Checking Website                    object
Predicted label                            bool
True/target label                        object
Altered image                            object
AI image                                 object
Tools used                               object
Model justification                      object
Ground truth justification               object
Possible reasons for wrong prediction    object
Possible Solution                        object
Other findings                           object
dtype: object


,Sample_index,Claim,Fact-Checking Website,Predicted label,True/target label,Altered image,AI image,Tools used,Model justification,Ground truth justification,Possible reasons for wrong prediction,Possible Solution,Other findings
0,1,image:2 This image shows the ruins after a 7.9...,AFP Factcheck,True,FALSE,No,No,"Reverse Image Search, Web Search",The image depicts the aftermath of the 2025 My...,"However, a reverse image search for the first ...",- Model did not compare the actual image with ...,"- E.g., extra_plan_rules= ""Prioritize using th...",NaN
1,8,<image:8> This image shows Israel launching se...,AFP Factcheck,True,FALSE,No,No,"Reverse Image Search, Web Search",The image depicts the aftermath of Israeli air...,"In reality, the image shows a different Israel...",- Model did not compare the actual image with ...,"- E.g., extra_plan_rules= ""Prioritize using th...",NaN
2,9,<image:9> This image shows an Israeli missile ...,AFP Factcheck,True,FALSE,No,No,"Reverse Image Search, Web Search","The claim is accurate. On October 26, 2024, Is...",A reverse image search and subsequent keyword ...,- Model did not compare the actual image with ...,"- E.g., extra_plan_rules= ""Prioritize using th...",NaN
3,17,A US-flagged vessel has been attacked in the G...,AFP Factcheck,True,FALSE,Yes,No,Web Search (2x),"The claim is accurate. On July 9, 2024, a U.S....",But a combination of reverse image and keyword...,- The image of the claim is missing and has be...,- Dataset correction,NaN
4,26,<image:19> This image is an authentic screensh...,Reuters,True,FALSE,Yes,No,Web Search,The claim is supported by evidence from a web ...,That remark and the fake screenshot were origi...,- Web search: Bad results from web search. The...,- Manipulation Detection Tool Use?\n- Prompt E...,- The report only has two pages and does not s...


![qualitative_analysis_part1](model1_run1_(2025-07-20_09-52%20default)/own_qualitative_analysis/model1_run1_qualitative_analysis_false_predictions_part1_n13_visualization.png)



In [22]:
pd.set_option('display.max_colwidth', None)
df_qualitative_analysis_part_1

,Sample_index,Claim,Fact-Checking Website,Predicted label,True/target label,Altered image,AI image,Tools used,Model justification,Ground truth justification,Possible reasons for wrong prediction,Possible Solution,Other findings
0,1,"image:2 This image shows the ruins after a 7.9-magnitude earthquake in Myanmar on March 28, 2025.",AFP Factcheck,True,FALSE,No,No,"Reverse Image Search, Web Search","The image depicts the aftermath of the 2025 Myanmar earthquake, which occurred on March 28, 2025, with a magnitude of 7.7-7.9, as confirmed by search results from sources like Wikipedia ([https://en.wikipedia.org/wiki/2025_Myanmar_earthquake](https://en.wikipedia.org/wiki/2025_Myanmar_earthquake)).","However, a reverse image search for the first photo in the post led to a TikTok video shared February 20 by a user based in Gaza. The clip shows the same destroyed building (archived link). A photo from the Turkish Anadolu news agency shows the same scene. ""A view of the destruction in Jabalia camp after Israeli forces withdrew from the area following the ceasefire in Gaza on February 05, 2025,"" the caption says.",- Model did not compare the actual image with the RIS/web search results \n- The actual image was not compared with images from search results/the internet\n- No images from the search are shown in the report; maybe no were scraped?\n- Maybe just focused on web search results?,"- E.g., extra_plan_rules= ""Prioritize using the results from image-based verification methods like image_search and reverse_search.”",NaN
1,8,<image:8> This image shows Israel launching several waves of revengeful attacks on Iranian military targets on October 26,AFP Factcheck,True,FALSE,No,No,"Reverse Image Search, Web Search","The image depicts the aftermath of Israeli airstrikes on Iranian military targets on October 26, 2024, as confirmed by the web search results. The strikes were in retaliation for earlier attacks and targeted military sites in Iran, Iraq, and Syria.","In reality, the image shows a different Israeli bombardment in Lebanon. Reverse image searches on Google found a corresponding image published by Lebanese photographer Nabil Ismail alongside ten other images on his Instagram post (archived link). On the night of October 6, the Israeli Defense Forces (IDF) issued warnings that specified their targets in Beirut’s southern suburb (archived link). Citing official Lebanese media, AFP reported that four Israeli strikes struck south Beirut on October 6, shortly after calls by Israel's army for residents to evacuate the Hezbollah stronghold which has been bombarded for several days (archived link). AFP reported on Israel's October 6 strike on Beirut, showing a different scene of the attack (archived link).",- Model did not compare the actual image with the RIS/web search results \n- The actual image was not compared with images from search results/the internet\n- No images from the search are shown in the report; maybe no were scraped?\n- Seems like the model only used/focused on the results from the web search,"- E.g., extra_plan_rules= ""Prioritize using the results from image-based verification methods like image_search and reverse_search.”",NaN
2,9,<image:9> This image shows an Israeli missile attack on Iran in the early hours of October 26,AFP Factcheck,True,FALSE,No,No,"Reverse Image Search, Web Search","The claim is accurate. On October 26, 2024, Israel launched airstrikes on military targets in Iran, as reported by sources such as [CNN](https://www.cnn.com/world/live-news/israel-iran-strikes-lebanon-gaza-war-10-26-24) and the [Associated Press](https://apnews.com/article/israel-palestinians-hamas-war-lebanon-hezbollah-iran-news-10-26-2024-9c9f366c71c508e6dd0ee74cff8400d2).","A reverse image search and subsequent keyword searches on Google found the falsely shared photos were taken by AFP photographer Atta Kenare and Vahid Ahmadi, a photographer for Iran's Tasnim news agency, on June 2, 2021. Both photos are available in AFP's archiv

#### Additional comments:

- First category: id's -> 1, 8, 9, 12, evtl. 96

    - id: 1
        - most likely reasons for false prediction:
            - Reverse Image Search (RIS) was not performed (the report only shows the image in the category "Evidence from reverse_search", but no other evidence from the internet)
            - As a result, the model only used the evidence from web search ("Myanmar earthquake March 28 2025") for the prediction. The web search only looks up sources that talk about the Myanmar earthquake. Thus, the web search is not linked to the claim image.

        - looking into the "log.txt" file for further debugging:
            - The model actually performed a RIS and found the image on three different Chinese and Taiwanese websites, which report that the image is falsely associated with the earthquake and is related to Gaza. 
            - The model also categorizes the evidence from all three websites as useful results.
            - After the RIS, the model performed a web search.
            - It's not clear, why the model did not use the evidence from the RIS for the prediction and why the evidence is also not listed/reported in the final "report.pdf", but is visible in the "log.txt" file.


    - id: 8
        - j

    

#### Qualitative Analyis Part 2 (n=18): 

- Prediction = Misleading
- Ground Truth Label = False

In [24]:
df_qualitative_analysis_part_2 = pd.read_csv("model1_run1_(2025-07-20_09-52 default)/own_qualitative_analysis/model1_run1_qualitative_analysis_false_predictions_part2_n18.csv")
print(len(df_qualitative_analysis_part_2))
print(df_qualitative_analysis_part_2.dtypes)
df_qualitative_analysis_part_2.head()

18
Sample_index                              int64
Claim                                    object
Fact-Checking Website                    object
Predicted label                          object
True/target label                          bool
True label is wrong                      object
Problem with true label                  object
Prediction actually true                 object
Altered Image                            object
AI image                                 object
Tools used                               object
Model justification                      object
Ground truth justification               object
Possible reasons for wrong prediction    object
Possible Solution                        object
dtype: object


,Sample_index,Claim,Fact-Checking Website,Predicted label,True/target label,True label is wrong,Problem with true label,Prediction actually true,Altered Image,AI image,Tools used,Model justification,Ground truth justification,Possible reasons for wrong prediction,Possible Solution
0,0,"<image:1> This image shows General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, which has died in the Gaza Strip in April 2025.",AFP Factcheck,MISLEADING,False,Yes,"- Label should have been “Misleading”, because the image is taken out of context",Yes,No,No,Reverse Image Search,"The image in the claim is a photograph of two soldiers in Babil Province, Iraq, taken in 2007, as evidenced by the [Wikimedia Commons entry](https://en.wikipedia.org/wiki/File:Two_19th_Group_Special_Forces_Soldiers_in_Babil_Province_Iraq.jpg). The claim incorrectly identifies the individuals and the location, making it misleading.","A reverse image search reveals the photograph was uploaded to Wikimedia Commons, a website that collects free-use images, with a description that says that it was taken in 2007 in Iraq's Babil province (archived here). Wikimedia Commons credits a blog post from the US National Guard as the source of the image. While the site has since been deleted, an archived version available via the Internet Archive says that the men in the image are two sergeants named Robbie Doman and Christopher W. Sullenberger. Doman, standing in the middle of the photograph, received media coverage in 2017 when he appeared on the NBC television show ""American Ninja Warrior"" after being injured in an explosion in 2008 (archived here, and here). Several US media outlets published images of Doman from his service in Iraq, which are similar to those used in the social media posts (archived here).",NaN,NaN
1,4,"<image:5> This image shows the U.S. bombarding Yemen on December 22, 2024 in reprisal after Yemen launched an attack on Israel's capital.",AFP Factcheck,MISLEADING,False,Yes,"- Label should have been “Misleading”, because the image is taken out of context",Yes,No,No,"Reverse Image Search, Web Search","The claim is misleading. While the U.S. did strike targets in Yemen on December 21, 2024, as reported by the [Times of Israel](https://www.timesofisrael.com/us-strikes-houthi-targets-in-yemen-takes-down-drones-over-red-sea/), the provided evidence does not confirm that the image depicts this specific event. The image's source is Getty Images, credited to Anadolu, but the search results do not explicitly link the image to the U.S. strikes on December 22, 2024.","But a reverse image search on Google found the same picture published by photo agency Getty Images on November 12, 2018 (archived link). Its caption read: ""GAZA CITY, GAZA - NOVEMBER 12: Smoke and flames blow up above the Hamas-run Al-Aqsa TV during Israeli airstrikes to positions in Gaza on November 12, 2018.”",NaN,NaN
2,5,"<image:6> This image shows an Israeli helicopter shot down by Hezbollah amid clashes in Southern Lebanon on November 23, 2024.",AFP Factcheck,MISLEADING,False,Yes,"- Label should have been “Misleading”, because the image is taken out of context",Yes,No,No,Reverse Image Search,"The image is not of an Israeli helicopter shot down in Southern Lebanon on November 23, 2024. Reverse image searches reveal that the image is related to events in Gaza in 2019, as reported by [The Times of Israel](https://www.timesofisrael.com/hamas-tried-to-down-an-idf-helicopter-during-fighting-this-month-report/), [PesaCheck](https://pesacheck.org/archive/2024/12/03), and [Arab News PK](https://www.arabnews.pk/node/1472406/middle-east).","A reverse image search on Google led to the same photo used in a report published by The Times of Israel newspaper on May 20, 2019 (archived link). The photo is credited to AFP photographer Mohammed Abed, and its caption read: ""An Israeli helicopter shoots flares over the Gaza Strip hours after a rocket from the Palestinian enclave hit a house near Tel 

![qualitative_analysis_part1](model1_run1_(2025-07-20_09-52%20default)/own_qualitative_analysis/model1_run1_qualitative_analysis_false_predictions_part2_n18_visualization.png)



In [25]:
df_qualitative_analysis_part_2["True label is wrong"].value_counts()

True label is wrong
Yes    11
No      7
Name: count, dtype: int64

In [26]:
# Print the sample indices of those claims where the true/ground truth label is actually wrong

sample_gt_label_wrong = df_qualitative_analysis_part_2[df_qualitative_analysis_part_2["True label is wrong"] == "Yes"]
sample_gt_label_wrong_indices = sample_gt_label_wrong["Sample_index"].to_list()

In [27]:
sample_gt_label_wrong_df_results = df_gaza_israel_merged[df_gaza_israel_merged["id"].isin(sample_gt_label_wrong_indices)]
sample_gt_label_wrong_df_results

## Let's look at the website distribution
print(f"{sample_gt_label_wrong_df_results["Website"].value_counts()}\n")

## Let's look at the claim type distribution
print(f"{sample_gt_label_wrong_df_results["Text_Only_Claim"].value_counts()}\n")
print(f"{sample_gt_label_wrong_df_results["Normal_Image"].value_counts()}\n")
print(f"{sample_gt_label_wrong_df_results["AI_Generated_Image"].value_counts()}\n")
print(f"{sample_gt_label_wrong_df_results["Altered_Image"].value_counts()}\n")

Website
AFP Factcheck         5
logicallyfacts.com    3
Reuters               1
checkyourfact.com     1
misbar.com            1
Name: count, dtype: int64

Text_Only_Claim
False    9
True     2
Name: count, dtype: int64

Normal_Image
True     8
False    3
Name: count, dtype: int64

AI_Generated_Image
False    11
Name: count, dtype: int64

Altered_Image
False    10
True      1
Name: count, dtype: int64



In [28]:
sample_gt_label_wrong_df_results

,Unnamed: 0,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Original_Claim_Website,Original_Claim_Only,...,Label,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,predicted,justification,correct
0,0,0,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.42JE3T2,False claims about US general being killed in Gaza spread online,2025-04-06 0:00:00,2025-04-17 0:00:00,Israeli-Palestinian conflict,"""BREAKING: Reports confirm the death of General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, in the Gaza Strip,"" says an April 6, 2025 X post with more than 30,000 likes.","""BREAKING: Reports confirm the death of General John Pagri, Commander of the Special Tasks Battalion in the U.S. Army, in the Gaza Strip.""",...,False,"A reverse image search reveals the photograph was uploaded to Wikimedia Commons, a website that collects free-use images, with a description that says that it was taken in 2007 in Iraq's Babil province (archived here). Wikimedia Commons credits a blog post from the US National Guard as the source of the image. While the site has since been deleted, an archived version available via the Internet Archive says that the men in the image are two sergeants named Robbie Doman and Christopher W. Sullenberger. Doman, standing in the middle of the photograph, received media coverage in 2017 when he appeared on the NBC television show ""American Ninja Warrior"" after being injured in an explosion in 2008 (archived here, and here). Several US media outlets published images of Doman from his service in Iraq, which are similar to those used in the social media posts (archived here).",False,True,False,False,Manual,MISLEADING,"The image in the claim is a photograph of two soldiers in Babil Province, Iraq, taken in 2007, as evidenced by the [Wikimedia Commons entry](https://en.wikipedia.org/wiki/File:Two_19th_Group_Special_Forces_Soldiers_in_Babil_Province_Iraq.jpg). The claim incorrectly identifies the individuals and the location, making it misleading.\n",False
4,4,4,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36RF6BZ,"Picture shows Israel's 2018 Gaza attack, not aftermath of US strike in Yemen",2024-12-22 0:00:00,2024-12-26 0:00:00,Israeli-Palestinian conflict,"""Americans bombarded Yemen in reprisal after Yemen launched an attack on Israel's capital,"" read a Thai-language Facebook post on December 22, 2024.","""Americans bombarded Yemen in reprisal after Yemen launched an attack on Israel's capital.""",...,False,"But a reverse image search on Google found the same picture published by photo agency Getty Images on November 12, 2018 (archived link). Its caption read: ""GAZA CITY, GAZA - NOVEMBER 12: Smoke and flames blow up above the Hamas-run Al-Aqsa TV during Israeli airstrikes to positions in Gaza on November 12, 2018.""",False,True,False,False,Manual,MISLEADING,"The claim is misleading. While the U.S. did strike targets in Yemen on December 21, 2024, as reported by the [Times of Israel](https://www.timesofisrael.com/us-strikes-houthi-targets-in-yemen-takes-down-drones-over-red-sea/), the provided evidence does not confirm that the image depicts this specific event. The image's source is Getty Images, credited to Anadolu, but the search results do not explicitly link the image to the U.S. strikes on December 22, 2024.\n",False
5,5,5,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.36NF2UW,"Photo shows Israeli helicopter firing flares over Gaza, not 'shot down in Lebanon'",2024-11-24 0:00:00,2024-12-02 0:00:00,Israeli-Palestinian conflict,"""Lebanon shot down its enemy's helicopter,"" read a Thai-language Facebook post shared on November 24, 2024.","""Lebanon shot down its enemy's helicopter.""",...,False,"A reverse image search on Google led to the same photo used in a report published by The Times of Israel newspaper on May 20, 2019 (archived link). The photo is credited to AFP photographer Mohammed Abed, and its 

#### Qualitative Analyis Part 3 (n=5): 

- Prediction = NEI
- Ground Truth Label = False

In [128]:
df_qualitative_analysis_part_3 = df_gaza_israel_merged.loc[(df_gaza_israel_merged["predicted"]=="NEI") & (df_gaza_israel_merged["Label"] == "False")]
df_qualitative_analysis_part_3

,Unnamed: 0,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Original_Claim_Website,Original_Claim_Only,...,Label,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,predicted,justification,correct
32,32,32,Reuters,https://www.reuters.com/fact-check/false-rumor...,Fact Check: False rumor said Israeli Air Force...,2024-11-28 0:00:00,2024-12-05 0:00:00,"""Israel Fact Check""","One of the social media posts, sharing the fal...","Commander of the Israeli Air Force, Major Gene...",...,False,A false rumor that Israeli Air Force chief Tom...,True,False,False,False,Manual,NEI,The claim cannot be verified. While multiple s...,False
49,49,49,Snopes,https://www.snopes.com/fact-check/kamala-harri...,"Kamala Harris Said, 'Our Support for Israel Wi...",2024-07-22 0:00:00,2024-07-25 0:00:00,"""Gaza""","On July 22, 2024, Vice President Kamala Harris...","""On July 22, 2024, Vice President Kamala Harri...",...,False,"While Harris did not use the above words, she ...",True,False,False,False,Manual,NEI,The claim cannot be verified. While Kamala Har...,False
57,57,57,misbar.com,https://www.misbar.com/en/factcheck/2025/04/13...,"Contrary to Trolls, Baby Sham Lost Her Arm and...",2025-04-13 00:00:00+00:00,2025-04-13 00:00:00+00:00,Gaza,NaN,Baby Sham did not lose her arm in an Israeli a...,...,False,Misbar’s investigative team spoke with Nahed ...,False,True,False,False,API,NEI,The claim is partially verifiable. While the i...,False
79,79,79,misbar.com,https://www.misbar.com/en/factcheck/2024/09/26...,This Photo Of Ramat David Is AI-Generated,2024-09-26 00:00:00+00:00,2024-09-26 00:00:00+00:00,Israel,NaN,Despite strict military censorship in Israel A...,...,False,The image was not shared by any credible medi...,False,False,True,False,API,NEI,"The claim cannot be verified because, while th...",False
97,98,98,logicallyfacts.com,https://www.logicallyfacts.com/en/fact-check/f...,AI image of Dwayne Johnson shared to claim tha...,2024-07-20 00:00:00+00:00,2024-07-11 00:00:00+00:00,Israel,NaN,The image shows Dwayne Johnson wearing Israeli...,...,False,A close look at the image showed several disc...,False,False,True,False,API,NEI,The available evidence confirms that Dwayne Jo...,False


#### Qualitative Analyis Part 4 (n=4): 

- Prediction = False
- Ground Truth Label = Misleading

In [129]:
df_qualitative_analysis_part_4 = df_gaza_israel_merged.loc[(df_gaza_israel_merged["predicted"]=="FALSE") & (df_gaza_israel_merged["Label"] == "Misleading")]
df_qualitative_analysis_part_4

,Unnamed: 0,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Original_Claim_Website,Original_Claim_Only,...,Label,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,predicted,justification,correct
24,24,24,Reuters,https://www.reuters.com/fact-check/unrwa-added...,Fact Check: UNRWA added black ribbon to its lo...,2024-10-18 0:00:00,2024-10-24 0:00:00,"""Gaza Fact Check""","Social media accounts shared, a photo of the U...","""UNRWA has placed a black band over its profil...",...,Misleading,Misleading. The black ribbon on the UNRWA logo...,False,True,False,False,Manual,FALSE,The claim is demonstrably false. Multiple fact...,False
66,66,66,misbar.com,https://www.misbar.com/en/factcheck/2025/03/05...,This Image Does Not Show Zelenskyy Signing Bom...,2025-03-05 00:00:00+00:00,2025-03-05 00:00:00+00:00,Israel,NaN,Ukrainian President Volodymyr Zelenskyy signin...,...,Misleading,Image of Zelenskyy at a Plant Producing Ammun...,False,True,False,False,API,FALSE,The image depicts Ukrainian President Volodymy...,False
68,68,68,misbar.com,https://www.misbar.com/en/factcheck/2025/03/01...,Image of 2021 Israeli Airstrike on Gaza Shared...,2025-03-01 00:00:00+00:00,2025-03-01 00:00:00+00:00,Israel,NaN,NOW: Israel is violently bombing the surroundi...,...,Misleading,2021 Israeli Airstrike on Gaza. Misbar's team...,False,True,False,False,API,FALSE,The image in the claim is associated with an a...,False
75,75,75,misbar.com,https://www.misbar.com/en/factcheck/2024/10/15...,This Image Shows Tents Engulfed in Flames Afte...,2024-10-15 00:00:00+00:00,2024-10-15 00:00:00+00:00,Israel,NaN,BREAKING: The UK has announced sanctions again...,...,Misleading,Israeli Airstrikes Spark Deadly Fire at Al-Aq...,False,True,False,False,API,FALSE,The claim is false. While the web search confi...,False


#### Qualitative Analyis Part 5 (n=3): 

- Prediction = NEI
- Ground Truth Label = Misleading

In [130]:
df_qualitative_analysis_part_5 = df_gaza_israel_merged.loc[(df_gaza_israel_merged["predicted"]=="NEI") & (df_gaza_israel_merged["Label"] == "Misleading")]
df_qualitative_analysis_part_5

,Unnamed: 0,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Original_Claim_Website,Original_Claim_Only,...,Label,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,predicted,justification,correct
61,61,61,misbar.com,https://www.misbar.com/en/factcheck/2024/07/15...,Photo of A Woman From South Lebanon Shared in ...,2024-07-15 00:00:00+00:00,2024-07-15 00:00:00+00:00,Gaza,NaN,Mercava Pride Zionist Industry Laundry clothes...,...,Misleading,2000 Photo of Israeli Tank Left Behind in Sou...,False,True,False,False,API,NEI,The provided evidence does not verify the clai...,False
69,69,69,misbar.com,https://www.misbar.com/en/factcheck/2025/02/27...,This Photo Is Not Related To Israeli Forces’ R...,2025-02-27 00:00:00+00:00,2025-02-27 00:00:00+00:00,Israel,NaN,Israeli occupation forces enter the town of Ta...,...,Misleading,Old Photo Shows Israeli Occupation Forces in ...,False,True,False,False,API,NEI,The claim cannot be fully verified. While ther...,False
93,93,93,misbar.com,https://www.misbar.com/en/factcheck/2024/07/23...,Old Photo of Crowded Ben Gurion Airport Resurf...,2024-07-23 00:00:00+00:00,2024-07-23 00:00:00+00:00,Israel,NaN,A photo shows Israelis fleeing the airport aft...,...,Misleading,Old Photo of Ben Gurion Airport Shared After ...,False,True,False,False,API,NEI,The claim cannot be verified. While a Houthi a...,False


#### Qualitative Analyis Part 6 (n=2): 

- Prediction = Misleading
- Ground Truth Label = NEI

In [132]:
df_qualitative_analysis_part_6 = df_gaza_israel_merged.loc[(df_gaza_israel_merged["predicted"]=="MISLEADING") & (df_gaza_israel_merged["Label"] == "NEI")]
df_qualitative_analysis_part_6

,Unnamed: 0,id,Website,Article_URL,Headline,Claim_Date,Review_Date,Query/Keyword,Original_Claim_Website,Original_Claim_Only,...,Label,Context/Label_Explanation,Text_Only_Claim,Normal_Image,AI_Generated_Image,Altered_Image,Data_Collection_Type,predicted,justification,correct
15,15,15,AFP Factcheck,https://factcheck.afp.com/doc.afp.com.367P9XU,No evidence Montreal mayor authorized pro-Pale...,2024-07-24 0:00:00,2024-08-06 0:00:00,Israeli-Palestinian conflict,"""So the Mayor is using taxpayer dollars to inc...","""So the Mayor is using taxpayer dollars to inc...",...,NEI,Some news headlines implied the boards were ha...,False,True,False,False,Manual,MISLEADING,The image depicts a traffic sign in Montreal d...,False
20,20,20,Reuters,https://www.reuters.com/fact-check/no-evidence...,Fact Check: No evidence US spent $50 million o...,2025-01-28 0:00:00,2025-01-30 0:00:00,"""Gaza Fact Check""","Posts online, sharing a clip of Leavitt’s rema...",“WILD: Karoline Leavitt EXPOSED that $50 MILLI...,...,NEI,Statements from the Trump administration sayin...,True,False,False,False,Manual,MISLEADING,The claim is misleading. While the White House...,False
